In [1]:
import src.data as data

data.update.DataUpdater.update_data()
data.update.DataUpdater.preprocess_data(True)

Update Files
Sun May  5 19:06:33 2024 : Updated ~ data\DataBase\DB_information\calendar.feather Done! Cost 0.05 Secs
Sun May  5 19:06:33 2024 : Updated ~ data\DataBase\DB_information\description.feather Done! Cost 0.04 Secs
Sun May  5 19:06:33 2024 : Updated ~ data\DataBase\DB_information\st.feather Done! Cost 0.02 Secs
Sun May  5 19:06:36 2024 : Updated ~ data\DataBase\DB_information\industry.feather Done! Cost 2.53 Secs
Sun May  5 19:06:40 2024 : Updated ~ data\DataBase\DB_information\concepts.feather Done! Cost 4.55 Secs
Sun May  5 19:06:41 2024 : Updated ~ data\DataBase\DB_models\longcl_exp\2024\longcl_exp.20240426.feather Done! Cost 0.11 Secs
Sun May  5 19:06:41 2024 : Updated ~ data\DataBase\DB_models\longcl_exp\2024\longcl_exp.20240429.feather Done! Cost 0.09 Secs
Sun May  5 19:06:53 2024 : download since!
Connection and Factor preparation finished!
Sun May  5 19:06:53 2024 : sellside/haitong.hf_factors from 20240501 to 20240505, total 1 periods
Start sellside/haitong.hf_factors

AttributeError: 'NoneType' object has no attribute 'tolist'

In [1]:
import src.data as data
data.update.DataUpdater.preprocess_data(True)

predict is True , Data Processing start!
3 datas :['y', 'trade_day', 'trade_30m']
y blocks loading start!
labels blocks reading ret10_lag DataBase's......  cost 0.58 secs
labels blocks reading ret20_lag DataBase's......  cost 0.49 secs
labels blocks merging......  cost 0.05 secs
models blocks reading risk_exp DataBase's......  cost 1.86 secs
models blocks merging......  cost 0.27 secs
y blocks loading cost 3.25 secs
y blocks process......  cost 2.67 secs
y blocks masking......  cost 0.10 secs
y blocks saving ......  cost 0.08 secs
y blocks norming......  cost 0.00 secs
Sun May  5 19:57:06 2024 : y finished! Cost 6.10 Seconds
trade_day blocks loading start!
trade blocks reading day DataBase's......  cost 0.75 secs
trade_day blocks merging......  cost 0.00 secs
trade_day blocks loading cost 0.76 secs
trade_day blocks process......  cost 0.06 secs
trade_day blocks masking......  cost 0.07 secs
trade_day blocks saving ......  cost 0.06 secs
trade_day blocks norming......  cost 0.00 secs
Su

In [1]:
import src.data as data
data.update.DataUpdater.preprocess_data(False)

IndexError: string index out of range

In [1]:
from src.interface import Predictor

model_preds = [
    Predictor('gru_day'    , 'swalast' , 0 , 'gru_day_V0') ,
    Predictor('gruRTN_day' , 'swalast' , 0 , 'gruRTN_day_V0') ,
    Predictor('gruRES_day' , 'swalast' , 0 , 'gruRES_day_V0')
]
[md.get_df().deploy() for md in model_preds]

Load  2 DataBlocks ...... cost 0.05 secs
Align 2 DataBlocks ...... cost 0.11 secs
Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
20231201 new
[       secid      date   gru_day
0          1  20240411  1.728767
1          2  20240411  2.449085
2          4  20240411 -1.272199
3          6  20240411 -0.489944
4          7  20240411  3.209286
...      ...       ...       ...
5098  688799  20240411  3.239002
5099  688800  20240411  1.156059
5100  688819  20240411  2.454775
5101  688981  20240411  0.749774
5102  689009  20240411  0.414008

[5103 rows x 3 columns],        secid      date   gru_day
0          1  20240412  1.506502
1          2  20240412  4.490141
2          4  20240412 -2.611845
3          6  20240412 -0.614345
4          7  20240412  3.103704
...      ...       ...       ...
5098  688799  20240412  2.735331
5099  688800  20240412 -1.545895
5100  688819  20240412  2.390810
5101  688981  20240412  1.832989
5102  689009  20240412  0.479365

[5103 rows x 3 colu

[None, None, None]

In [2]:
from src.interface import ModelTestor

mt = ModelTestor.new('ts_mixer')
mt.try_metrics()

Load  2 DataBlocks ...

... cost 0.22 secs
Align 2 DataBlocks ...... cost 0.21 secs
Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
x shape is torch.Size([5064, 30, 6])
y shape is torch.Size([5064, 1])
Test Forward Success
metrics :  Metrics.MetricOutput(loss=tensor(1.0688, grad_fn=<AddBackward0>), score=-0.015420470386743546, loss_item=1.068833589553833, penalty=0.0, losses=tensor(1.0688, grad_fn=<ExpBackward0>))
Test Metrics Success
